In [ ]:
# Variational Quantum Linear Solver
# based on https://qiskit.org/textbook/ch-paper-implementations/vqls.html
# VQLS paper: https://arxiv.org/abs/1909.05820 

In [1]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, transpile, assemble
import math
import random
import numpy as np
from scipy.optimize import minimize

In [2]:
# Fixed Hardware Ansatz 
def apply_fixed_ansatz(qubits, parameters):

    for iz in range (0, len(qubits)):
        circ.ry(parameters[0][iz], qubits[iz])

    circ.cz(qubits[0], qubits[1])
    circ.cz(qubits[2], qubits[0])

    for iz in range (0, len(qubits)):
        circ.ry(parameters[1][iz], qubits[iz])

    circ.cz(qubits[1], qubits[2])
    circ.cz(qubits[2], qubits[0])

    for iz in range (0, len(qubits)):
        circ.ry(parameters[2][iz], qubits[iz])

circ = QuantumCircuit(3)
apply_fixed_ansatz([0, 1, 2], [[1, 1, 1], [1, 1, 1], [1, 1, 1]])
circ.draw()

┌───────┐      ┌───────┐      ┌───────┐
q_0: ┤ Ry(1) ├─■──■─┤ Ry(1) ├────■─┤ Ry(1) ├
     ├───────┤ │  │ ├───────┤    │ ├───────┤
q_1: ┤ Ry(1) ├─■──┼─┤ Ry(1) ├─■──┼─┤ Ry(1) ├
     ├───────┤    │ ├───────┤ │  │ ├───────┤
q_2: ┤ Ry(1) ├────■─┤ Ry(1) ├─■──■─┤ Ry(1) ├
     └───────┘      └───────┘      └───────┘

In [3]:
# Creates the Hadamard test

def had_test(gate_type, qubits, auxiliary_index, parameters):

    circ.h(auxiliary_index)

    apply_fixed_ansatz(qubits, parameters)

    for ie in range (0, len(gate_type[0])):
        if (gate_type[0][ie] == 1):
            circ.cz(auxiliary_index, qubits[ie])

    for ie in range (0, len(gate_type[1])):
        if (gate_type[1][ie] == 1):
            circ.cz(auxiliary_index, qubits[ie])
    
    circ.h(auxiliary_index)
    
circ = QuantumCircuit(4)
had_test(gate_type=[[0, 1, 0], [0, 0, 1]], qubits=[1, 2, 3], auxiliary_index=0, 
         parameters=[[1, 1, 1], [1, 1, 1], [1, 1, 1]])
circ.draw()

┌───┐                                      ┌───┐
q_0: ──┤ H ├─────────────────────────────────■──■─┤ H ├
     ┌─┴───┴─┐      ┌───────┐      ┌───────┐ │  │ └───┘
q_1: ┤ Ry(1) ├─■──■─┤ Ry(1) ├────■─┤ Ry(1) ├─┼──┼──────
     ├───────┤ │  │ ├───────┤    │ ├───────┤ │  │      
q_2: ┤ Ry(1) ├─■──┼─┤ Ry(1) ├─■──┼─┤ Ry(1) ├─■──┼──────
     ├───────┤    │ ├───────┤ │  │ ├───────┤    │      
q_3: ┤ Ry(1) ├────■─┤ Ry(1) ├─■──■─┤ Ry(1) ├────■──────
     └───────┘      └───────┘      └───────┘

In [4]:
# Creates controlled anstaz for calculating |<b|psi>|^2 with a Hadamard test

def control_fixed_ansatz(qubits, parameters, auxiliary, reg):

    for i in range (0, len(qubits)):
        circ.cry(parameters[0][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    circ.ccx(auxiliary, qubits[1], 4)
    circ.cz(qubits[0], 4)
    circ.ccx(auxiliary, qubits[1], 4)

    circ.ccx(auxiliary, qubits[0], 4)
    circ.cz(qubits[2], 4)
    circ.ccx(auxiliary, qubits[0], 4)

    for i in range (0, len(qubits)):
        circ.cry(parameters[1][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    circ.ccx(auxiliary, qubits[2], 4)
    circ.cz(qubits[1], 4)
    circ.ccx(auxiliary, qubits[2], 4)

    circ.ccx(auxiliary, qubits[0], 4)
    circ.cz(qubits[2], 4)
    circ.ccx(auxiliary, qubits[0], 4)

    for i in range (0, len(qubits)):
        circ.cry(parameters[2][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

q_reg = QuantumRegister(5)
circ = QuantumCircuit(q_reg)
control_fixed_ansatz([1, 2, 3], [[1, 1, 1], [1, 1, 1], [1, 1, 1]], 0, q_reg)
circ.draw(fold=-1)

q0_0: ────■────────■────────■──────■───────■────■───────■──────■────────■────────■──────■───────■────■───────■──────■────────■────────■────
      ┌───┴───┐    │        │      │       │    │       │  ┌───┴───┐    │        │      │       │    │       │  ┌───┴───┐    │        │    
q0_1: ┤ Ry(1) ├────┼────────┼──────┼───■───┼────■───────■──┤ Ry(1) ├────┼────────┼──────┼───────┼────■───────■──┤ Ry(1) ├────┼────────┼────
      └───────┘┌───┴───┐    │      │   │   │    │       │  └───────┘┌───┴───┐    │      │       │    │       │  └───────┘┌───┴───┐    │    
q0_2: ─────────┤ Ry(1) ├────┼──────■───┼───■────┼───────┼───────────┤ Ry(1) ├────┼──────┼───■───┼────┼───────┼───────────┤ Ry(1) ├────┼────
               └───────┘┌───┴───┐  │   │   │    │       │           └───────┘┌───┴───┐  │   │   │    │       │           └───────┘┌───┴───┐
q0_3: ──────────────────┤ Ry(1) ├──┼───┼───┼────┼───■───┼────────────────────┤ Ry(1) ├──■───┼───■────┼───■───┼────────────────────┤ Ry(1) ├
                        └───────┘┌─┴─┐ │ ┌─┴─┐┌─┴─┐ │ ┌─┴─┐                  └───────┘┌─┴─┐ │ ┌─┴─┐┌─┴─┐ │ ┌─┴─┐                  └───────┘
q0_4: ───────────────────────────┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├───────────────────────────┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├───────────────────────────
                                 └───┘   └───┘└───┘   └───┘                           └───┘   └───┘└───┘   └───┘

In [5]:
# we pick U as 3x Hadamard
def control_b(auxiliary, qubits):

    for ia in qubits:
        circ.ch(auxiliary, ia)

circ = QuantumCircuit(4)
control_b(0, [1, 2, 3])
circ.draw()

q_0: ──■────■────■──
     ┌─┴─┐  │    │  
q_1: ┤ H ├──┼────┼──
     └───┘┌─┴─┐  │  
q_2: ─────┤ H ├──┼──
          └───┘┌─┴─┐
q_3: ──────────┤ H ├
               └───┘

In [6]:
# Create the controlled Hadamard test, for calculating <psi|psi>

def special_had_test(gate_type, qubits, auxiliary_index, parameters, reg):

    circ.h(auxiliary_index)

    control_fixed_ansatz(qubits, parameters, auxiliary_index, reg)

    for ty in range (0, len(gate_type)):
        if (gate_type[ty] == 1):
            circ.cz(auxiliary_index, qubits[ty])


    control_b(auxiliary_index, qubits)
    
    circ.h(auxiliary_index)

q_reg = QuantumRegister(5)
circ = QuantumCircuit(q_reg)
special_had_test([[0, 0, 0], [0, 0, 1]], [1, 2, 3], 0, [[1, 1, 1], [1, 1, 1], [1, 1, 1]], q_reg)
circ.draw()

┌───┐                                                              »
q1_0: ┤ H ├────■────────■────────■──────■───────■────■───────■──────■────»
      └───┘┌───┴───┐    │        │      │       │    │       │  ┌───┴───┐»
q1_1: ─────┤ Ry(1) ├────┼────────┼──────┼───■───┼────■───────■──┤ Ry(1) ├»
           └───────┘┌───┴───┐    │      │   │   │    │       │  └───────┘»
q1_2: ──────────────┤ Ry(1) ├────┼──────■───┼───■────┼───────┼───────────»
                    └───────┘┌───┴───┐  │   │   │    │       │           »
q1_3: ───────────────────────┤ Ry(1) ├──┼───┼───┼────┼───■───┼───────────»
                             └───────┘┌─┴─┐ │ ┌─┴─┐┌─┴─┐ │ ┌─┴─┐         »
q1_4: ────────────────────────────────┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├─────────»
                                      └───┘   └───┘└───┘   └───┘         »
«                                                                             »
«q1_0: ────■────────■──────■───────■────■───────■──────■────────■────────■────»
«          │        │      │       │    │       │  ┌───┴───┐    │        │    »
«q1_1: ────┼────────┼──────┼───────┼────■───────■──┤ Ry(1) ├────┼────────┼────»
«      ┌───┴───┐    │      │       │    │       │  └───────┘┌───┴───┐    │    »
«q1_2: ┤ Ry(1) ├────┼──────┼───■───┼────┼───────┼───────────┤ Ry(1) ├────┼────»
«      └───────┘┌───┴───┐  │   │   │    │       │           └───────┘┌───┴───┐»
«q1_3: ─────────┤ Ry(1) ├──■───┼───■────┼───■───┼────────────────────┤ Ry(1) ├»
«               └───────┘┌─┴─┐ │ ┌─┴─┐┌─┴─┐ │ ┌─┴─┐                  └───────┘»
«q1_4: ──────────────────┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├───────────────────────────»
«                        └───┘   └───┘└───┘   └───┘                           »
«                     ┌───┐
«q1_0: ──■────■────■──┤ H ├
«      ┌─┴─┐  │    │  └───┘
«q1_1: ┤ H ├──┼────┼───────
«      └───┘┌─┴─┐  │       
«q1_2: ─────┤ H ├──┼───────
«           └───┘┌─┴─┐     
«q1_3: ──────────┤ H ├─────
«                └───┘     
«q1_4: ────────────────────
«

In [20]:
# Implements the entire cost function on the quantum circuit
# this code works w/ the state vectors arther than shots

def calculate_cost_function(parameters):
    
    global opt

    overall_sum_1 = 0
    
    parameters = [parameters[0:3], parameters[3:6], parameters[6:9]]

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):
            print('CFC i,j',i,j)
            global circ

            qctl = QuantumRegister(5)
            qc = ClassicalRegister(5)
            circ = QuantumCircuit(qctl, qc)

            backend = Aer.get_backend('aer_simulator')
            
            multiply = coefficient_set[i]*coefficient_set[j]

            had_test([gate_set[i], gate_set[j]], [1, 2, 3], 0, parameters)

            circ.save_statevector()
            t_circ = transpile(circ, backend)
            qobj = assemble(t_circ)
            job = backend.run(qobj)

            result = job.result()
            outputstate = np.real(result.get_statevector(circ, decimals=100))
            o = outputstate
            print('passA o',len(o),o)
            m_sum = 0
            for l in range (0, len(o)):
                if (l%2 == 1):
                    n = o[l]**2
                    m_sum+=n
                    print('ss',l,o[l],m_sum,1-(2*m_sum))

            overall_sum_1+=multiply*(1-(2*m_sum))

    overall_sum_2 = 0

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            multiply = coefficient_set[i]*coefficient_set[j]
            mult = 1

            for extra in range(0, 2):

                qctl = QuantumRegister(5)
                qc = ClassicalRegister(5)
                circ = QuantumCircuit(qctl, qc)

                backend = Aer.get_backend('aer_simulator')

                if (extra == 0):
                    special_had_test(gate_set[i], [1, 2, 3], 0, parameters, qctl)
                if (extra == 1):
                    special_had_test(gate_set[j], [1, 2, 3], 0, parameters, qctl)

                circ.save_statevector()    
                t_circ = transpile(circ, backend)
                qobj = assemble(t_circ)
                job = backend.run(qobj)

                result = job.result()
                outputstate = np.real(result.get_statevector(circ, decimals=100))
                o = outputstate

                m_sum = 0
                for l in range (0, len(o)):
                    if (l%2 == 1):
                        n = o[l]**2
                        m_sum+=n
                mult = mult*(1-(2*m_sum))

            overall_sum_2+=multiply*mult

    print('psi|psi',overall_sum_1,' b|psi',overall_sum_2)
    print(1-float(overall_sum_2/overall_sum_1))

    return 1-float(overall_sum_2/overall_sum_1)

In [21]:
#The final step is to actually use this code to solve a real linear system: A= 0.458Z3+ 0.55*I
#    In order to minimize the cost function, we use the COBYLA optimizer method,

coefficient_set = [0.55, 0.45]
gate_set = [[0, 0, 0], [0, 0, 1]]
numIter=5
out = minimize(calculate_cost_function, x0=[float(random.randint(0,3000))/1000 for i in range(0, 9)], 
               method="COBYLA", options={'maxiter':numIter})
print('J:end of minimizer', out)

out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9]]

circ = QuantumCircuit(3, 3)
apply_fixed_ansatz([0, 1, 2], out_f)
circ.save_statevector()

backend = Aer.get_backend('aer_simulator')
t_circ = transpile(circ, backend)
qobj = assemble(t_circ)
job = backend.run(qobj)

result = job.result()
o = result.get_statevector(circ, decimals=10)

a1 = coefficient_set[1]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a2 = coefficient_set[0]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1]])
a3 = np.add(a1, a2)

b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

print((b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o)))))**2)

CFC i,j 0 0
passA o 32 [-0.20417228  0.         -0.18288085  0.         -0.36048461  0.
 -0.34382602  0.         -0.19217933  0.         -0.15739359  0.
 -0.58648677  0.         -0.52060581  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
ss 1 0.0 0.0 1.0
ss 3 0.0 0.0 1.0
ss 5 0.0 0.0 1.0
ss 7 0.0 0.0 1.0
ss 9 0.0 0.0 1.0
ss 11 0.0 0.0 1.0
ss 13 0.0 0.0 1.0
ss 15 0.0 0.0 1.0
ss 17 0.0 0.0 1.0
ss 19 0.0 0.0 1.0
ss 21 0.0 0.0 1.0
ss 23 0.0 0.0 1.0
ss 25 0.0 0.0 1.0
ss 27 0.0 0.0 1.0
ss 29 0.0 0.0 1.0
ss 31 0.0 0.0 1.0
CFC i,j 0 1
passA o 32 [-2.04172280e-01 -1.95790234e-17 -1.82880853e-01 -4.18000661e-18
 -3.60484615e-01 -6.29011277e-18 -3.43826018e-01  2.77885500e-17
 -4.82551376e-17 -1.92179330e-01 -2.77139770e-17 -1.57393588e-01
 -6.43496994e-17 -5.86486774e-01 -5.42597703e-17 -5.20605813e-01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000

b|psi 0.49058612301755106 psi|psi 0.13609111566855384
0.7225948528028685
CFC i,j 0 0
passA o 32 [-0.35823126  0.         -0.31802685  0.         -0.21112356  0.
 -0.2298571   0.         -0.45796838  0.         -0.37857696  0.
 -0.43175501  0.         -0.36558708  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
ss 1 0.0 0.0 1.0
ss 3 0.0 0.0 1.0
ss 5 0.0 0.0 1.0
ss 7 0.0 0.0 1.0
ss 9 0.0 0.0 1.0
ss 11 0.0 0.0 1.0
ss 13 0.0 0.0 1.0
ss 15 0.0 0.0 1.0
ss 17 0.0 0.0 1.0
ss 19 0.0 0.0 1.0
ss 21 0.0 0.0 1.0
ss 23 0.0 0.0 1.0
ss 25 0.0 0.0 1.0
ss 27 0.0 0.0 1.0
ss 29 0.0 0.0 1.0
ss 31 0.0 0.0 1.0
CFC i,j 0 1
passA o 32 [-3.58231256e-01 -8.18244485e-19 -3.18026846e-01 -4.04121845e-17
 -2.11123555e-01  1.09568593e-17 -2.29857101e-01 -2.33995677e-17
 -6.47538986e-17 -4.57968380e-01 -7.16928551e-17 -3.78576956e-01
 -2.25211342e-17 -4.31755013e-01 -1.96322011e-17 -3.

b|psi 0.502588248922899 psi|psi 0.21919047554299068
0.563876640544741
CFC i,j 0 0
passA o 32 [-0.19090331  0.         -0.43250828  0.         -0.05424186  0.
 -0.36568612  0.         -0.19956047  0.         -0.56497291  0.
 -0.21860864  0.         -0.48271678  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.        ]
ss 1 0.0 0.0 1.0
ss 3 0.0 0.0 1.0
ss 5 0.0 0.0 1.0
ss 7 0.0 0.0 1.0
ss 9 0.0 0.0 1.0
ss 11 0.0 0.0 1.0
ss 13 0.0 0.0 1.0
ss 15 0.0 0.0 1.0
ss 17 0.0 0.0 1.0
ss 19 0.0 0.0 1.0
ss 21 0.0 0.0 1.0
ss 23 0.0 0.0 1.0
ss 25 0.0 0.0 1.0
ss 27 0.0 0.0 1.0
ss 29 0.0 0.0 1.0
ss 31 0.0 0.0 1.0
CFC i,j 0 1
passA o 32 [-1.90903308e-01 -2.91887095e-17 -4.32508285e-01 -7.59136021e-17
 -5.42418592e-02 -4.89555532e-19 -3.65686118e-01 -5.19125194e-17
 -4.20869640e-17 -1.99560465e-01 -2.25712232e-16 -5.64972909e-01
 -4.33312907e-17 -2.18608640e-01 -1.19132755e-16 -4.827

gate_type: [[0, 0, 1], [0, 0, 1]]
0.012660528156429263
J:end of minimizer      fun: 0.012396611922237066
   maxcv: 0.0
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 200
  status: 2
 success: False
       x: array([3.14373378, 1.64393623, 2.31270616, 2.44154603, 1.48288088,
       2.2627108 , 1.9645516 , 1.69921923, 2.97555323])
(0.9876033880685445-0j)

#As you can see, our cost function has achieved a fairly low value of 0.012, and when we calculate our classical
#cost function, we get 0.988, which agrees perfectly what we measured, the vectors  \psi0 and b

In [21]:
b

array([0.35355339, 0.35355339, 0.35355339, 0.35355339, 0.35355339,
       0.35355339, 0.35355339, 0.35355339])

In [22]:
a3.dot(o)

array([ 0.38259084+0.j,  0.23665533+0.j,  0.4576645 +0.j,  0.3408739 +0.j,
        0.03981846+0.j,  0.03665728+0.j, -0.03295611+0.j, -0.02653857+0.j])

In [23]:
o

Statevector([ 0.38259084+0.j,  0.23665533+0.j,  0.4576645 +0.j,
              0.3408739 +0.j,  0.39818455+0.j,  0.36657281+0.j,
             -0.32956114+0.j, -0.26538572+0.j],
            dims=(2, 2, 2))


In [ ]:
#Implements the entire cost function on the quantum circuit (sampling, 100000 shots)

def calculate_cost_function(parameters):

    global opt

    overall_sum_1 = 0
    
    parameters = [parameters[0:3], parameters[3:6], parameters[6:9]]

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            global circ

            qctl = QuantumRegister(5)
            qc = ClassicalRegister(1)
            circ = QuantumCircuit(qctl, qc)

            backend = Aer.get_backend('aer_simulator')
            
            multiply = coefficient_set[i]*coefficient_set[j]

            had_test([gate_set[i], gate_set[j]], [1, 2, 3], 0, parameters)

            circ.measure(0, 0)

            t_circ = transpile(circ, backend)
            qobj = assemble(t_circ, shots=10000)
            job = backend.run(qobj)

            result = job.result()
            outputstate = result.get_counts(circ)

            if ('1' in outputstate.keys()):
                m_sum = float(outputstate["1"])/100000
            else:
                m_sum = 0

            overall_sum_1+=multiply*(1-2*m_sum)

    overall_sum_2 = 0

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            multiply = coefficient_set[i]*coefficient_set[j]
            mult = 1

            for extra in range(0, 2):

                qctl = QuantumRegister(5)
                qc = ClassicalRegister(1)
                
                circ = QuantumCircuit(qctl, qc)

                backend = Aer.get_backend('aer_simulator')

                if (extra == 0):
                    special_had_test(gate_set[i], [1, 2, 3], 0, parameters, qctl)
                if (extra == 1):
                    special_had_test(gate_set[j], [1, 2, 3], 0, parameters, qctl)

                circ.measure(0, 0)

                t_circ = transpile(circ, backend)
                qobj = assemble(t_circ, shots=10000)
                job = backend.run(qobj)

                result = job.result()
                outputstate = result.get_counts(circ)

                if ('1' in outputstate.keys()):
                    m_sum = float(outputstate["1"])/100000
                else:
                    m_sum = 0

                mult = mult*(1-2*m_sum)
            
            overall_sum_2+=multiply*mult
            
    print(1-float(overall_sum_2/overall_sum_1))

    return 1-float(overall_sum_2/overall_sum_1)

In [ ]:
coefficient_set = [0.55, 0.225, 0.225]
gate_set = [[0, 0, 0], [0, 1, 0], [0, 0, 1]]

out = minimize(calculate_cost_function, x0=[float(random.randint(0,3000))/1000 for i in range(0, 9)], method="COBYLA", options={'maxiter':200})
print(out)

out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9]]

circ = QuantumCircuit(3, 3)
apply_fixed_ansatz([0, 1, 2], out_f)
circ.save_statevector()

backend = Aer.get_backend('aer_simulator')

t_circ = transpile(circ, backend)
qobj = assemble(t_circ)
job = backend.run(qobj)

result = job.result()
o = result.get_statevector(circ, decimals=10)

a1 = coefficient_set[2]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a0 = coefficient_set[1]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,-1,0,0,0,0,0], [0,0,0,-1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1]])
a2 = coefficient_set[0]*np.array([[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1]])

a3 = np.add(np.add(a2, a0), a1)

b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

print((b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o)))))**2)

gate_type: [[0, 0, 1], [0, 0, 1]]
0.029056506965673745
     fun: 0.029056506965673745
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 101
  status: 1
 success: True
       x: array([ 3.97749195, -0.3196544 ,  1.45782757,  3.81477212,  2.16994105,
        4.02622184,  2.51930974,  4.00136043,  3.87112544])
(0.7528501534683052+0j)

#So as you can see, not amazing, our solution is still off by a fairly significant margin ( 30% ? error isn't awful,
#but ideally, we want it to be much closer to 0). Again, I think this is due to the optimizer itself, 
#not the actual quantum circuit. I will be making an update to this Notebook once I figure out how to correct 
#this problem (likely with the introduction of a noisy optimizer, as I previously mentioned).

